# 🎓 ViT 전이학습 (Transfer Learning)

> **학습 목표**: 사전학습된 ViT 모델을 제조 불량 데이터로 Fine-tuning하여 고성능 분류기를 구축합니다.

---

## 📋 학습 내용

1. ✅ 사전학습 모델 로드 (ViT-Base-Patch16-224)
2. ✅ 데이터셋 준비 (Train 70%, Val 15%, Test 15%)
3. ✅ 분류 헤드 교체 (4-class 제조 불량)
4. ✅ Fine-tuning 전략 (Learning Rate Scheduling)
5. ✅ 학습 모니터링 (Loss, Accuracy Curves)
6. ✅ 성능 평가 (Confusion Matrix, Per-class Metrics)
7. ✅ Feature 시각화 (t-SNE Embedding)

**소요 시간**: 약 50분  
**난이도**: ⭐⭐⭐⭐ (고급)  
**사전 지식**: 딥러닝 기초, 전이학습 개념, PyTorch 입문

---

## 🔧 Step 1: 라이브러리 Import

In [ ]:
# 딥러닝 프레임워크
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# Hugging Face Transformers
try:
    import transformers
    from transformers import (
        ViTForImageClassification,
        ViTImageProcessor,
        Trainer,
        TrainingArguments
    )
    print(f"✅ Transformers 버전: {transformers.__version__}")
except ImportError:
    print("⚠️ Transformers 설치 필요: pip install transformers")

# 이미지 처리
from PIL import Image
import torchvision.transforms as transforms

# 데이터 분석
import pandas as pd
import numpy as np

# 평가 메트릭
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, precision_recall_fscore_support
)
from sklearn.manifold import TSNE

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 유틸리티
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (14, 6)

# 시드 고정 (재현성)
torch.manual_seed(42)
np.random.seed(42)

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   메모리: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

print("\n✅ 라이브러리 로드 완료!")

## 📂 Step 2: 데이터셋 준비

**데이터 구조**: `defect_images/` 디렉토리에 클래스별 이미지 저장  
**분할**: Train 70%, Validation 15%, Test 15%

In [ ]:
# 데이터 디렉토리 설정
# 1순위: KAMP 실제 데이터 (시지트로닉스 이미지 - 6개 불량 유형)
# 2순위: 로컬 data/defect_images 폴더
# 3순위: 합성 데이터 자동 생성 (FileNotFoundError 방지)

import tempfile

kamp_dir = Path('../../dataset/part2-2/시지트로닉스_이미지')
local_dir = Path('../data/defect_images')

if kamp_dir.exists() and any(kamp_dir.iterdir()):
    data_dir = kamp_dir
    # KAMP 실제 클래스: AREA, DISTRIBUTION, FAIL, NEEDLE, PASS, SCATCH
    classes = sorted([d.name for d in kamp_dir.iterdir() if d.is_dir()])
    data_source = "KAMP 실제 데이터 (시지트로닉스)"
elif local_dir.exists() and any(local_dir.rglob('*.png')) or local_dir.exists() and any(local_dir.rglob('*.jpg')):
    data_dir = local_dir
    classes = sorted([d.name for d in local_dir.iterdir() if d.is_dir()])
    if not classes:
        classes = ['normal', 'scratch', 'contamination', 'damage']
    data_source = "로컬 데이터"
else:
    # ⚠️ CRITICAL: 합성 데이터 생성 (원본 코드는 여기서 FileNotFoundError 발생)
    print("⚠️ 실제 데이터를 찾을 수 없습니다. 합성 학습 데이터를 생성합니다...")
    data_dir = Path(tempfile.mkdtemp(prefix='vit_defect_'))
    classes = ['normal', 'scratch', 'contamination', 'damage']
    data_source = f"합성 데이터 (자동 생성: {data_dir})"
    
    for cls in classes:
        cls_dir = data_dir / cls
        cls_dir.mkdir(parents=True, exist_ok=True)
        # 클래스당 30장의 합성 이미지 생성 (총 120장)
        for i in range(30):
            img_array = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
            # 클래스별 시각적 차이 부여
            if cls == 'scratch':
                # 대각선 스크래치 패턴
                for offset in range(-2, 3):
                    coords = np.arange(224)
                    img_array[coords, np.clip(coords + offset, 0, 223)] = [255, 255, 255]
            elif cls == 'contamination':
                # 원형 얼룩 패턴
                y, x = np.ogrid[-112:112, -112:112]
                mask = x**2 + y**2 <= np.random.randint(20, 50)**2
                img_array[mask] = [80, 60, 40]
            elif cls == 'damage':
                # 불규칙 노이즈 패턴
                noise = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
                region = slice(50, 174), slice(50, 174)
                img_array[region] = noise[region]
            
            img = Image.fromarray(img_array)
            img.save(cls_dir / f'{cls}_{i:03d}.png')
    
    print(f"   ✅ 합성 데이터 생성 완료: {len(classes)}개 클래스 × 30장 = 120장")

num_classes = len(classes)

print(f"📂 데이터 경로: {data_dir}")
print(f"📦 데이터 소스: {data_source}")
print(f"\n📊 클래스 정보 ({num_classes}개):")
for idx, cls in enumerate(classes):
    cls_dir = data_dir / cls
    if cls_dir.exists():
        image_count = len(list(cls_dir.glob('*.png'))) + len(list(cls_dir.glob('*.jpg')))
        print(f"   [{idx}] {cls}: {image_count}개 이미지")
    else:
        print(f"   [{idx}] {cls}: 디렉토리 없음")

print(f"\n✅ 데이터 디렉토리 준비 완료!")

In [ ]:
# Custom Dataset 클래스
class DefectDataset(Dataset):
    """제조 불량 이미지 데이터셋"""
    
    def __init__(self, data_dir, classes, processor, augment=False):
        self.data_dir = Path(data_dir)
        self.classes = classes
        self.processor = processor
        self.augment = augment
        
        # 이미지 파일 수집
        self.image_paths = []
        self.labels = []
        
        for idx, cls in enumerate(classes):
            cls_dir = self.data_dir / cls
            image_files = list(cls_dir.glob('*.jpg')) + list(cls_dir.glob('*.png'))
            
            for img_path in image_files:
                self.image_paths.append(img_path)
                self.labels.append(idx)
        
        print(f"📦 데이터셋 생성: {len(self)}개 샘플")
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # 이미지 로드
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        
        # Data Augmentation (Training only)
        if self.augment:
            transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(15),
                transforms.ColorJitter(brightness=0.2, contrast=0.2),
            ])
            image = transform(image)
        
        # ViT Processor 적용 (Resize + Normalize)
        inputs = self.processor(images=image, return_tensors='pt')
        pixel_values = inputs['pixel_values'].squeeze(0)
        
        return {
            'pixel_values': pixel_values,
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ViT Image Processor 로드
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
print(f"✅ ViT Processor 로드 완료!")
print(f"   입력 크기: {processor.size}")
print(f"   정규화: mean={processor.image_mean}, std={processor.image_std}")

In [ ]:
# 전체 데이터셋 생성
full_dataset = DefectDataset(
    data_dir=data_dir,
    classes=classes,
    processor=processor,
    augment=False  # 전체 데이터에는 augmentation 적용 안함
)

# Train / Val / Test 분할 (70% / 15% / 15%)
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"\n📊 데이터 분할:")
print(f"   Train: {len(train_dataset):,}개 ({len(train_dataset)/total_size*100:.1f}%)")
print(f"   Validation: {len(val_dataset):,}개 ({len(val_dataset)/total_size*100:.1f}%)")
print(f"   Test: {len(test_dataset):,}개 ({len(test_dataset)/total_size*100:.1f}%)")

# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\n✅ DataLoader 생성 완료! (배치 크기: {batch_size})")

## 🤖 Step 3: 사전학습 모델 로드 및 분류 헤드 교체

**모델**: ViT-Base-Patch16-224 (ImageNet 사전학습)  
**전략**: 분류 헤드만 교체하여 4-class 제조 불량 분류

In [ ]:
print("🔄 사전학습 ViT 모델 로드 중...")

# ViT 모델 로드 (ImageNet 사전학습)
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=num_classes,  # 분류 헤드 교체 (1000 → 4)
    ignore_mismatched_sizes=True  # 크기 불일치 무시
)

# 모델을 GPU로 이동
model = model.to(device)

print(f"\n✅ 모델 로드 완료!")
print(f"   모델명: ViT-Base-Patch16-224")
print(f"   파라미터 수: {sum(p.numel() for p in model.parameters()):,}")
print(f"   학습 가능 파라미터: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# 모델 구조 확인
print(f"\n📊 모델 구조:")
print(f"   ViT Encoder: 12 layers")
print(f"   Hidden size: 768")
print(f"   Attention heads: 12")
print(f"   Patch size: 16×16")
print(f"   Image size: 224×224")
print(f"   분류 헤드: {num_classes} classes")

## 🎓 Step 4: Fine-tuning 설정

**학습률 전략**: Warmup + Cosine Decay  
**최적화**: AdamW with weight decay

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir='../outputs/vit_finetuned',
    
    # 학습 설정
    num_train_epochs=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    
    # 최적화
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_ratio=0.1,  # 10% warmup
    lr_scheduler_type='cosine',  # Cosine decay
    
    # 평가
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    
    # 로깅
    logging_dir='../outputs/logs',
    logging_steps=10,
    
    # 기타
    seed=42,
    fp16=torch.cuda.is_available(),  # Mixed precision (GPU only)
    dataloader_num_workers=0,  # macOS + Jupyter 환경 호환 (fork 충돌 방지)
)

print("📋 Training Arguments:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Learning Rate: {training_args.learning_rate}")
print(f"   Batch Size: {training_args.per_device_train_batch_size}")
print(f"   Weight Decay: {training_args.weight_decay}")
print(f"   Warmup Ratio: {training_args.warmup_ratio}")
print(f"   LR Scheduler: {training_args.lr_scheduler_type}")
print(f"   FP16: {training_args.fp16}")

In [ ]:
# 평가 메트릭 함수
def compute_metrics(eval_pred):
    """정확도 및 F1 계산"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # 정확도
    accuracy = accuracy_score(labels, predictions)
    
    # Precision, Recall, F1 (macro average)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='macro'
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("\n✅ Trainer 생성 완료!")
print(f"   총 학습 스텝: {len(train_loader) * training_args.num_train_epochs:,}")
print(f"   Warmup 스텝: {int(len(train_loader) * training_args.num_train_epochs * training_args.warmup_ratio):,}")

## 🚀 Step 5: 모델 학습

**주의**: 실제 학습은 GPU 환경에서 수행 권장 (약 10-20분 소요)

In [ ]:
print("🔄 Fine-tuning 시작...\n")

# 학습 실행
train_result = trainer.train()

# 학습 결과
print("\n✅ Fine-tuning 완료!")
print(f"\n📊 학습 결과:")
print(f"   최종 Loss: {train_result.training_loss:.4f}")
print(f"   학습 시간: {train_result.metrics['train_runtime']:.1f}초")
print(f"   평균 속도: {train_result.metrics['train_samples_per_second']:.1f} samples/sec")

# 모델 저장
trainer.save_model('../outputs/vit_finetuned_best')
print(f"\n💾 모델 저장: ../outputs/vit_finetuned_best")

## 📊 Step 6: 학습 곡선 시각화

**Loss & Accuracy** 추이 확인

In [ ]:
# 학습 로그 추출
log_history = trainer.state.log_history

# Train/Val Loss 추출
train_loss = [log['loss'] for log in log_history if 'loss' in log]
eval_loss = [log['eval_loss'] for log in log_history if 'eval_loss' in log]
eval_accuracy = [log['eval_accuracy'] for log in log_history if 'eval_accuracy' in log]

epochs = range(1, len(eval_loss) + 1)

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Loss Curve
axes[0].plot(train_loss, label='Train Loss', linewidth=2, color='skyblue')
axes[0].plot(
    np.linspace(0, len(train_loss), len(eval_loss)),
    eval_loss,
    label='Val Loss',
    linewidth=2,
    color='coral',
    marker='o'
)
axes[0].set_xlabel('Training Steps', fontweight='bold')
axes[0].set_ylabel('Loss', fontweight='bold')
axes[0].set_title('Loss Curve', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy Curve
axes[1].plot(epochs, eval_accuracy, linewidth=2, color='green', marker='o')
axes[1].set_xlabel('Epoch', fontweight='bold')
axes[1].set_ylabel('Accuracy', fontweight='bold')
axes[1].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_ylim([0, 1])
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/02_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📈 학습 곡선:")
print(f"   최종 Train Loss: {train_loss[-1]:.4f}")
print(f"   최종 Val Loss: {eval_loss[-1]:.4f}")
print(f"   최고 Val Accuracy: {max(eval_accuracy):.4f} (Epoch {np.argmax(eval_accuracy)+1})")

## 🎯 Step 7: 테스트 세트 평가

**Confusion Matrix & 클래스별 성능**

In [ ]:
print("🔄 테스트 세트 평가 중...\n")

# 테스트 예측
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=-1)
y_true = predictions.label_ids

# 전체 정확도
test_accuracy = accuracy_score(y_true, y_pred)
print(f"📊 테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

# 클래스별 성능
print(f"\n📋 Classification Report:")
report = classification_report(y_true, y_pred, target_names=classes, digits=4)
print(report)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Raw Counts
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=classes,
    yticklabels=classes,
    ax=axes[0]
)
axes[0].set_xlabel('Predicted', fontweight='bold')
axes[0].set_ylabel('Actual', fontweight='bold')
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')

# Normalized
sns.heatmap(
    cm_normalized,
    annot=True,
    fmt='.2%',
    cmap='Greens',
    xticklabels=classes,
    yticklabels=classes,
    ax=axes[1]
)
axes[1].set_xlabel('Predicted', fontweight='bold')
axes[1].set_ylabel('Actual', fontweight='bold')
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('../outputs/02_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 🎨 Step 8: Feature 시각화 (t-SNE)

**ViT의 마지막 Hidden State**를 t-SNE로 2D 시각화

In [ ]:
print("🔄 Feature Extraction 중...\n")

# Feature 추출 함수
def extract_features(model, dataloader, device):
    """ViT의 마지막 hidden state 추출"""
    model.eval()
    features = []
    labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            pixel_values = batch['pixel_values'].to(device)
            batch_labels = batch['labels'].cpu().numpy()
            
            # Forward pass (마지막 hidden state 추출)
            outputs = model.vit(
                pixel_values=pixel_values,
                output_hidden_states=True
            )
            
            # [CLS] token의 hidden state 사용
            cls_hidden_state = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            
            features.append(cls_hidden_state)
            labels.append(batch_labels)
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    return features, labels

# 테스트 세트에서 Feature 추출
test_features, test_labels = extract_features(model, test_loader, device)

print(f"✅ Feature Extraction 완료!")
print(f"   Feature Shape: {test_features.shape}")
print(f"   Feature Dimension: {test_features.shape[1]}")

In [ ]:
print("🔄 t-SNE 차원 축소 중... (약 1-2분 소요)\n")

# t-SNE 적용
tsne = TSNE(
    n_components=2,
    perplexity=30,
    n_iter=1000,
    random_state=42
)

features_2d = tsne.fit_transform(test_features)

print(f"✅ t-SNE 완료!")
print(f"   2D Embedding Shape: {features_2d.shape}")

# 시각화
plt.figure(figsize=(12, 10))

colors = ['blue', 'red', 'green', 'orange']
for idx, cls in enumerate(classes):
    mask = test_labels == idx
    plt.scatter(
        features_2d[mask, 0],
        features_2d[mask, 1],
        c=colors[idx],
        label=cls,
        alpha=0.6,
        s=50
    )

plt.xlabel('t-SNE Dimension 1', fontsize=12, fontweight='bold')
plt.ylabel('t-SNE Dimension 2', fontsize=12, fontweight='bold')
plt.title('ViT Feature Space (t-SNE)', fontsize=14, fontweight='bold', pad=20)
plt.legend(fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../outputs/02_tsne_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n💡 해석:")
print(f"   - 같은 클래스가 가까이 모여 있으면 특징 학습이 잘 된 것")
print(f"   - 다른 클래스가 명확히 분리되면 분류 성능이 좋음")
print(f"   - 클러스터 겹침이 있으면 혼동 가능성 존재")

## 💾 Step 9: 결과 저장

In [ ]:
# 출력 디렉토리 생성
output_dir = Path('../outputs')
output_dir.mkdir(exist_ok=True)

# 1. 학습 히스토리
history_df = pd.DataFrame({
    'epoch': epochs,
    'val_loss': eval_loss,
    'val_accuracy': eval_accuracy
})
history_file = output_dir / '02_training_history.csv'
history_df.to_csv(history_file, index=False, encoding='utf-8-sig')
print(f"✅ 학습 히스토리 저장: {history_file}")

# 2. 테스트 결과
test_results = pd.DataFrame({
    'class': classes,
    'precision': precision_recall_fscore_support(y_true, y_pred, average=None)[0],
    'recall': precision_recall_fscore_support(y_true, y_pred, average=None)[1],
    'f1_score': precision_recall_fscore_support(y_true, y_pred, average=None)[2],
    'support': precision_recall_fscore_support(y_true, y_pred, average=None)[3]
})
test_results_file = output_dir / '02_test_results.csv'
test_results.to_csv(test_results_file, index=False, encoding='utf-8-sig')
print(f"✅ 테스트 결과 저장: {test_results_file}")

# 3. Confusion Matrix
cm_df = pd.DataFrame(cm, index=classes, columns=classes)
cm_file = output_dir / '02_confusion_matrix.csv'
cm_df.to_csv(cm_file, encoding='utf-8-sig')
print(f"✅ Confusion Matrix 저장: {cm_file}")

# 4. t-SNE Features
tsne_df = pd.DataFrame({
    'tsne_dim1': features_2d[:, 0],
    'tsne_dim2': features_2d[:, 1],
    'label': test_labels,
    'class_name': [classes[label] for label in test_labels]
})
tsne_file = output_dir / '02_tsne_features.csv'
tsne_df.to_csv(tsne_file, index=False, encoding='utf-8-sig')
print(f"✅ t-SNE Features 저장: {tsne_file}")

print("\n🎉 모든 결과 저장 완료!")

---

## 🎯 학습 정리

### ✅ 완료한 내용

1. **사전학습 모델 활용** - ImageNet으로 사전학습된 ViT-Base 로드
2. **분류 헤드 교체** - 1000-class → 4-class 제조 불량 분류
3. **데이터 분할** - Train 70%, Validation 15%, Test 15%
4. **Fine-tuning 전략** - Learning Rate Scheduling (Warmup + Cosine Decay)
5. **학습 모니터링** - Loss & Accuracy Curves
6. **성능 평가** - Confusion Matrix, Per-class Metrics
7. **Feature 시각화** - t-SNE로 2D Embedding

---

### 💡 핵심 인사이트

#### 전이학습의 장점

**사전학습 모델 활용**:
- ✅ 적은 데이터로 고성능 달성 (수백~수천 장)
- ✅ 학습 시간 단축 (처음부터 학습 대비 10배 이상)
- ✅ 일반화 성능 향상 (ImageNet 지식 전이)

**Fine-tuning 전략**:
- **Learning Rate**: 사전학습보다 낮게 (2e-4 ~ 5e-5)
- **Warmup**: 초기 불안정성 방지 (10%)
- **Cosine Decay**: 학습률을 점진적으로 감소
- **Weight Decay**: 과적합 방지 (0.01)

---

#### ViT vs CNN 비교

**ViT의 장점**:
- ✅ 장거리 의존성 학습 (Global attention)
- ✅ 확장성 (더 큰 모델로 성능 향상)
- ✅ 해석 가능성 (Attention map 분석)

**ViT의 단점**:
- ❌ 많은 데이터 필요 (전이학습으로 해결)
- ❌ 계산 비용 높음 (GPU 필수)
- ❌ Inductive bias 부족 (CNN은 locality 내장)

---

#### 실무 적용 가이드

**데이터 크기별 전략**:
- **소규모 (<1K)**: 강한 Data Augmentation + Freeze 일부 레이어
- **중규모 (1K-10K)**: 전체 Fine-tuning (이 실습)
- **대규모 (>10K)**: Scratch 학습 고려

**성능 개선 팁**:
1. **Data Augmentation**: 제조 도메인 특화 (회전, 조명 변화)
2. **Class Imbalance**: 가중치 조정 또는 Oversampling
3. **Ensemble**: 여러 모델 결합으로 robustness 향상
4. **Test-Time Augmentation**: 예측 시 augmentation 적용

**배포 전 체크리스트**:
- ✅ 테스트 정확도 90% 이상
- ✅ 클래스별 성능 균형 (Recall 차이 <10%)
- ✅ 추론 속도 목표 달성 (<100ms/image)
- ✅ 모델 경량화 (Quantization, Pruning)

---

### 📚 다음 단계

- **Part 2-2**: YOLOv8 객체 탐지 Fine-tuning
- **Part 3-1**: 한국어 임베딩 및 RAG 시스템

### 🔗 참고 자료

- [ViT 논문](https://arxiv.org/abs/2010.11929) - An Image is Worth 16x16 Words
- [Hugging Face Transformers](https://huggingface.co/docs/transformers/)
- [Transfer Learning Guide](https://ruder.io/transfer-learning/)

---

*제조AI 교육 v12 Enhanced | Part 2-2 | 2025.02*